<a href="https://colab.research.google.com/github/LeoRyu18/AM2_T1/blob/main/AM2_T1_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1º Trabalho de Aprendizado de Máquina II - UFSCar 2020/2

---

**Trabalho referente a aplicação de métodos de Classificação multirrótulo em múltiplos datasets.**

**Professor**
> Diego Furtado Silva

**Participantes:**

>Leonardo Ryu Takaki -> lrtakaki@estudante.ufscar.br

>Augusto Rozendo Mends -> augustorm@estudante.ufscar.br

>Fernando Sassi Nunes -> fernando.nunes@estudante.ufscar.br

---

- Algoritmos
 - MLkNN
 - Relevância binária (BR)
 - Label Powerset (LP)




- Base de dados
 - Emotions
 - FoodTrucks
 - Birds
 -
 -

---

In [ ]:
!pip install scikit-multilearn

In [ ]:
# importações

import pandas as pd
import numpy as np
from matplotlib import pyplot as pyplot


import sklearn.metrics as metrics
from skmultilearn.adapt import MLkNN
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV


from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

---
## MLkNN

* Derivado de dois algoritmos monorrótulos: KNN e Naive Bayes;

* 1) KNN é aplicado, identificando os k vizinhos mais próximos do novo objeto na base de treinamento.

* 2) A Fórmula de Bayes é empregada para cada rótulo de classe lj presente no labelset destes k vizinhos para estimar se t deverá ou não ser rotulado com lj.



In [ ]:
def MLkNN_simples(X_train, y_train, X_test, y_test):
    classifier = MLkNN(k=3)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))

def MLkNN_GS(X_train, y_train, X_test, y_test):
    parameters = {'k': range(1,10), 's': [0.1, 0.3, 0.5, 0.7, 1.0]}

    model = GridSearchCV(MLkNN(), parameters, scoring='accuracy')
    model.fit(X_train, y_train)
    print("Melhores parâmetros =", model.best_params_,"\nMelhor score =", model.best_score_)

    predictions = model.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))

---
## Relevância binária (BR)

* Um classificador binário de rótulo único é treinado para cada classe.

* Cada classificador prevê a associação ou a não associação de uma classe para um objeto. A união de todas as classes previstas será a saída.

* Considera independência entre os rótulos.



In [ ]:
def BR_simples(X_train, y_train, X_test, y_test):
    classifier = BinaryRelevance( classifier = SVC(), require_dense = [False, True])
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))

def BR_GS(X_train, y_train, X_test, y_test):
    parameters = [{'classifier': [MultinomialNB()],
                   'classifier__alpha': [0.5, 0.7, 1.0],
                  },
                  {'classifier': [SVC()],
                   'classifier__kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                  },
                  {'classifier':[GaussianNB()],
                   'classifier__var_smoothing':[0.00000001, 0.000000001, 0.00000001],
                  },
                  {'classifier': [RandomForestClassifier()],
                   'classifier__criterion': ['gini', 'entropy'],
                   'classifier__n_estimators': [10, 20, 50],
                  },
                 ]
    model = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy')
    model.fit(X_train, y_train)
    print("Melhores parâmetros =", model.best_params_,"\nMelhor score =", model.best_score_)

    predictions = model.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))

---
## LP (Label Powerset)

* Essa abordagem leva em consideração as possíveis correlações entre os rótulos de classe.

* Considera cada combinação das classes como um único rótulo.

* Tem uma alta complexidade computacional, tendo em vista que as combinações distintas de rótulos cresce exponencialmente quando o número de classes aumenta, tornando o problema difícil.



In [ ]:
def LP_simples(X_train, y_train, X_test, y_test):
    classifier = ClassifierChain(classifier = RandomForestClassifier(n_estimators=100), require_dense = [False, True])
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))


def LP_GS(X_train, y_train, X_test, y_test):
    parameters = [{'classifier': [MultinomialNB()],
                   'classifier__alpha': [0.5, 0.7, 1.0],
                  },
                  {'classifier': [RandomForestClassifier()],
                   'classifier__criterion': ['gini', 'entropy'],
                   'classifier__n_estimators': [10, 20, 50],
                  },
                  {'classifier': [SVC()],
                   'classifier__kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                  },
                  {'classifier':[GaussianNB()],
                   'classifier__var_smoothing':[0.00000001, 0.000000001, 0.00000001],
                  },
                 ]
    model = GridSearchCV(LabelPowerset(), parameters, scoring='accuracy')
    model.fit(X_train, y_train)
    print("Melhores parâmetros =", model.best_params_,"\nMelhor score =", model.best_score_)

    predictions = model.predict(X_test)
    print("Hamming Loss =", metrics.hamming_loss(y_test, predictions))
    print("Acurácia =", metrics.accuracy_score(y_test, predictions))
    print("Precisão =", metrics.precision_score(y_test, predictions, average='micro'))
    print("Revocação =", metrics.recall_score(y_test, predictions, average='micro'))


---

# Datasets + aplicação dos algoritmos

## Emotions

Os humanos, por natureza, são afetados emocionalmente pela música. À medida que os bancos de dados de música aumentam de tamanho, a recuperação da música pela emoção se torna uma
tarefa importante para várias aplicações, como seleção de música em dispositivos móveis, sistemas de recomendação de música, programas de TV, rádio e musicoterapia.
A música pode evocar mais de uma emoção diferente ao mesmo tempo. Desta forma, queremos recuperar uma peça musical baseada em qualquer uma das emoções. A classificação e regressão de rótulo único não podem modelar essa multiplicidade, portanto, utilizamos alguns algoritmos de classificação multirrótulo.

Os features extraídos se enquadram em duas categorias: rítmica e timbre.

* Features Rítmicos:
As características rítmicas foram derivadas da extração das alterações periódicas de um histograma de batida. Um algoritmo que identifica picos foi utilizado e os dois picos mais altos foram selecionados para calcular as amplitudes,
seus BPMs (batidas por minuto) e a proporção de alta BPM e baixa BPM. Além disso, 3 recursos foram calculados somando os bins do histograma entre 40-90, 90-140 e 140-
250 BPMs respectivamente. Desta forma, temos 
8 features rítmicos.

* Features de timbre:
Coeficientes mel-cepstrais (MFCCs) são usados ​​para
reconhecimento de fala e modelagem musical. Para derivar features MFCCs, o sinal foi dividido em frames e o espectro de amplitude foi calculado para cada frame. Em seguida, seu logaritmo foi obtido e convertido para a escala de Mel. Finalmente, a
transformada discreta de cosseno foi aplicada. No Dataset selecionado, foram utilizados os primeiros 13 MFCCs.
Outro conjunto de 3 features relacionados a texturas de timbre foram extraídos da Transformada de Fourier de Curto Termo (FFT): Centróide espectral, rolloff espectral e fluxo espectral.
Para cada um dos 16 recursos mencionados acima (13 MFCCs e 3 FFT) foram calculadas a média, o desvio padrão (std),
desvio padrão médio (mean std) e desvio padrão
de desvio padrão (std std) em todos os frames. Isso levou a um total de 64 recursos de timbre.

Temos 6 classes: amazed-surprised, happy-pleased, relaxing-calm, quiet-still, sad-lonely, angry-fearful. Os exemplos foram classificados por três especialistas do sexo masculino de 20, 25 e 30 anos. Apenas as músicas com rotulagem completamente idêntica entre os especialistas foram mantidas para experimentação subsequente. Esse processo levou a um conjunto de dados com 593 músicas classificados.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/LeoRyu18/AM2_T1/main/emotions.csv?token=AHW73BL6QJI2YHTKQFMQHTDAKVXNU")
df.head()

,id,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,Mean_Acc1298_Mean_Mem40_MFCC_1,Mean_Acc1298_Mean_Mem40_MFCC_2,Mean_Acc1298_Mean_Mem40_MFCC_3,Mean_Acc1298_Mean_Mem40_MFCC_4,Mean_Acc1298_Mean_Mem40_MFCC_5,Mean_Acc1298_Mean_Mem40_MFCC_6,Mean_Acc1298_Mean_Mem40_MFCC_7,Mean_Acc1298_Mean_Mem40_MFCC_8,Mean_Acc1298_Mean_Mem40_MFCC_9,Mean_Acc1298_Mean_Mem40_MFCC_10,Mean_Acc1298_Mean_Mem40_MFCC_11,Mean_Acc1298_Mean_Mem40_MFCC_12,Mean_Acc1298_Std_Mem40_Centroid,Mean_Acc1298_Std_Mem40_Rolloff,Mean_Acc1298_Std_Mem40_Flux,Mean_Acc1298_Std_Mem40_MFCC_0,Mean_Acc1298_Std_Mem40_MFCC_1,Mean_Acc1298_Std_Mem40_MFCC_2,Mean_Acc1298_Std_Mem40_MFCC_3,Mean_Acc1298_Std_Mem40_MFCC_4,Mean_Acc1298_Std_Mem40_MFCC_5,Mean_Acc1298_Std_Mem40_MFCC_6,Mean_Acc1298_Std_Mem40_MFCC_7,Mean_Acc1298_Std_Mem40_MFCC_8,Mean_Acc1298_Std_Mem40_MFCC_9,Mean_Acc1298_Std_Mem40_MFCC_10,Mean_Acc1298_Std_Mem40_MFCC_11,Mean_Acc1298_Std_Mem40_MFCC_12,Std_Acc1298_Mean_Mem40_Centroid,Std_Acc1298_Mean_Mem40_Rolloff,Std_Acc1298_Mean_Mem40_Flux,Std_Acc1298_Mean_Mem40_MFCC_0,Std_Acc1298_Mean_Mem40_MFCC_1,Std_Acc1298_Mean_Mem40_MFCC_2,Std_Acc1298_Mean_Mem40_MFCC_3,Std_Acc1298_Mean_Mem40_MFCC_4,Std_Acc1298_Mean_Mem40_MFCC_5,Std_Acc1298_Mean_Mem40_MFCC_6,Std_Acc1298_Mean_Mem40_MFCC_7,Std_Acc1298_Mean_Mem40_MFCC_8,Std_Acc1298_Mean_Mem40_MFCC_9,Std_Acc1298_Mean_Mem40_MFCC_10,Std_Acc1298_Mean_Mem40_MFCC_11,Std_Acc1298_Mean_Mem40_MFCC_12,Std_Acc1298_Std_Mem40_Centroid,Std_Acc1298_Std_Mem40_Rolloff,Std_Acc1298_Std_Mem40_Flux,Std_Acc1298_Std_Mem40_MFCC_0,Std_Acc1298_Std_Mem40_MFCC_1,Std_Acc1298_Std_Mem40_MFCC_2,Std_Acc1298_Std_Mem40_MFCC_3,Std_Acc1298_Std_Mem40_MFCC_4,Std_Acc1298_Std_Mem40_MFCC_5,Std_Acc1298_Std_Mem40_MFCC_6,Std_Acc1298_Std_Mem40_MFCC_7,Std_Acc1298_Std_Mem40_MFCC_8,Std_Acc1298_Std_Mem40_MFCC_9,Std_Acc1298_Std_Mem40_MFCC_10,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BH_HighLowRatio,BHSUM1,BHSUM2,BHSUM3,amazed-suprised,happy-pleased,relaxing-calm,quiet-still,sad-lonely,angry-aggresive
0,1,0.034741,0.089665,0.091225,-73.302422,6.215179,0.615074,2.037160,0.804065,1.301409,0.558576,0.672063,0.783788,0.766640,0.458712,0.530384,0.812429,0.028851,0.129039,0.039614,5.762173,1.636819,1.170034,1.051511,0.764163,0.642705,0.617868,0.510265,0.566213,0.509149,0.477275,0.505073,0.463535,0.013519,0.050591,0.009025,8.156257,1.077167,0.624711,0.810244,0.399568,0.279947,0.314215,0.231439,0.345401,0.285389,0.210613,0.321896,0.290551,0.022774,0.095801,0.015057,4.748694,0.536378,0.296306,0.273210,0.175800,0.105508,0.168246,0.115849,0.136020,0.110514,0.100517,0.118630,0.094923,0.051035,68,0.014937,136,2,0.245457,0.105065,0.405399,0,1,1,0,0,0
1,2,0.081374,0.272747,0.085733,-62.584437,3.183163,-0.218145,0.163038,0.620251,0.458514,0.041426,0.308287,0.538152,0.594871,0.734332,0.415489,0.761508,0.066288,0.262370,0.034438,3.480874,1.596532,0.943803,0.804444,0.511229,0.498670,0.523039,0.480916,0.488657,0.483166,0.445187,0.415994,0.405593,0.013621,0.073041,0.010094,1.243981,0.829790,0.252972,0.347831,0.205087,0.168601,0.178009,0.144080,0.178703,0.146937,0.125580,0.128202,0.107007,0.020028,0.066940,0.029483,3.963534,0.382360,0.168389,0.117525,0.098341,0.087046,0.057991,0.059393,0.059457,0.053439,0.067684,0.070075,0.041565,0.295031,70,0.276366,140,2,0.343547,0.276366,0.710924,1,0,0,0,0,1
2,3,0.110545,0.273567,0.084410,-65.235325,2.794964,0.639047,1.281297,0.757896,0.489412,0.627636,0.469322,0.644336,0.441556,0.335964,0.290713,0.158538,0.082743,0.215373,0.035970,4.834742,1.213443,0.864034,0.909222,0.780572,0.550833,0.639740,0.573309,0.526312,0.562622,0.538407,0.492292,0.455562,0.029112,0.070433,0.008525,2.759906,0.592634,0.761852,0.568740,0.589827,0.281181,0.437752,0.479889,0.227320,0.296224,0.273855,0.191804,0.198025,0.038119,0.065427,0.029622,3.371796,0.430373,0.172862,0.177523,0.184333,0.095718,0.139323,0.109279,0.090650,0.117886,0.100852,0.079917,0.085821,0.1

In [ ]:
# Pré-processamento 

df = df.drop(columns=['id'])

class_pos = ["amazed-suprised", "happy-pleased", "relaxing-calm", "quiet-still", "sad-lonely", "angry-aggresive"]
X_train, X_test, y_train, y_test = train_test_split(df.drop(class_pos, axis = 1), df[class_pos],  test_size = 0.3, random_state = 1)

# Normalização
X_train = preprocessing.MinMaxScaler().fit_transform(X_train)
X_test = preprocessing.MinMaxScaler().fit_transform(X_test)
y_train = y_train.values
y_test = y_test.values

In [ ]:
# MLkNN
print("MLkNN simples:")
MLkNN_simples(X_train, y_train, X_test, y_test)

print("\nMLkNN com Grid Search:")
MLkNN_GS(X_train, y_train, X_test, y_test)

MLkNN simples:
Hamming Loss = 0.2144194756554307
Acurácia = 0.29213483146067415
Precisão = 0.6778115501519757
Revocação = 0.6445086705202312

MLkNN com Grid Search:
Melhores parâmetros = {'k': 3, 's': 0.1} 
Melhor score = 0.28674698795180725
Hamming Loss = 0.2144194756554307
Acurácia = 0.29213483146067415
Precisão = 0.6778115501519757
Revocação = 0.6445086705202312


In [ ]:
# BR
print("BR simples:")
BR_simples(X_train, y_train, X_test, y_test)

print("\nBR com Grid Search:")
BR_GS(X_train, y_train, X_test, y_test)

BR simples:
Hamming Loss = 0.18726591760299627
Acurácia = 0.2640449438202247
Precisão = 0.8173913043478261
Revocação = 0.5433526011560693

BR com Grid Search:
Melhores parâmetros = {'classifier': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'classifier__kernel': 'rbf'} 
Melhor score = 0.30120481927710846
Hamming Loss = 0.18726591760299627
Acurácia = 0.2640449438202247
Precisão = 0.8173913043478261
Revocação = 0.5433526011560693


In [ ]:
# LP
print("LP simples:")
LP_simples(X_train, y_train, X_test, y_test)

print("\nLP com Grid Search:")
LP_GS(X_train, y_train, X_test, y_test)

LP simples:
Hamming Loss = 0.19382022471910113
Acurácia = 0.2752808988764045
Precisão = 0.7791164658634538
Revocação = 0.5606936416184971

LP com Grid Search:
Melhores parâmetros = {'classifier': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'classifier__criterion': 'gini', 'classifier__n_estimators': 50} 
Melhor score = 0.3493975903614458
Hamming Loss = 0.21910112359550563
Acurácia = 0.3258426966292135
Precisão = 0.6647058823529411
Revocação = 0.653179190751445



---
## FoodTrucks

Este dataset (disponível em ) contém resultados de uma pesquisa conduzida em Natal, que visava determinar preferência por certos tipos de food trucks dependendo de fatores diversos. São 407 exemplos no total.

As features são:

* frequência(numérico): com que frequência uma pessoa come fora, de 1(raramente) a 5(diariamente ou quase diariamente)

* tempo (categórico): em que parte do dia costuma comer fora. Os possíveis valores são: manhã,almoço, tarde, happy hour e janta

* gasto (numérico): quanto um dado indivíduo costuma gastar, em reais. Os valores determinam os limites dos intervalos: 15,20,30,40,50.

* motivação (categórico): o que leva um indivíduo a escolher determinado foodtruck: propaganda, acaso, amigos, redes sociais, internet

* gosto, higiene, menu, apresentação, atendimento, ingredientes, lugar disponível,para viagem, variação de escolha, encontros, horários: quanto um dado indivíduo valoriza determinadas qualidades de um foodtruck, numa escala de 1 a 5.

* gênero(categórico)

* escolaridade(numérico): de sem estudo a phd numa escala de 1 a 5. Vale notar que alunos em processo de graduação são representados por 1.5

* renda média(numérico): em termos de salários mínimos, de 1 a 5, sendo 1 menos de 2 e 5 mais de 20

* trabalha (binário)

* estado civil (categórico)

* idade (numérico): agrupados em intervalos, sendo o primeiro <19, os subsequentes de 5 em 5, terminando em um intervalo 50+. Assume os valores de 1 a 8.

In [ ]:
df_food = pd.read_csv("https://raw.githubusercontent.com/LeoRyu18/AM2_T1/main/food.csv?token=AHW73BLI3WHUSEIWBNJPWKLAKVW3K")
df_food.head()

,frequency,time,expenses,motivation,taste,hygiene,menu,presentation,attendance,ingredients,place.to.sit,takeaway,variation,stop.strucks,schedule,gender,age.group,scholarity,average.income,has.work,marital.status,street_food,gourmet,italian_food,brazilian_food,mexican_food,chinese_food,japanese_food,arabic_food,snacks,healthy_food,fitness_food,sweets_desserts
0,2,dinner,30,friend,5,4,4,3,4,4,4,3,3,2,2,M,2,1.5,4,0,single,1,0,0,0,0,0,0,0,0,0,0,1
1,0,dinner,20,by_chance,5,2,4,4,4,4,5,2,3,1,4,F,3,2.0,5,1,married,1,0,1,0,1,0,0,0,0,0,0,1
2,1,afternoon,15,by_chance,5,2,2,5,3,5,2,2,2,2,1,M,2,1.5,4,1,single,1,0,0,1,0,0,0,0,0,0,0,0
3,0,lunch,40,friend,5,5,4,3,4,4,3,4,5,1,1,M,3,2.0,6,1,single,0,1,0,0,0,0,0,0,0,0,0,0
4,0,dinner,15,social_network,5,4,2,3,4,4,2,3,4,1,1,M,2,1.5,4,1,single,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
labels = ['italian_food','brazilian_food','mexican_food','chinese_food','japanese_food','arabic_food','snacks','healthy_food','fitness_food','sweets_desserts','gourmet','street_food']

y = df_food[labels]
X = df_food.drop(labels,axis=1)

In [ ]:
#encoding de features categóricos
X.replace({'dawn':1,'lunch':2,'afternoon':3,'happy_hour':4,'dinner':5
,'ads':1,'by_chance':2,'friend':3,'social_network':4,'web':5,
'F':1,'M':2,
'divorced':1,'married':2,'single':3
}, inplace=True)

# o único feature que já não vem normalizado numa escala de 1 a 5 é a renda. Não sei se isso é um oversight ou intencional por parte do estudo mas eu decidi normalizar

X_train = preprocessing.MinMaxScaler().fit_transform(X_train)
X_test = preprocessing.MinMaxScaler().fit_transform(X_test)

#Separação treino-teste
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42) 
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



In [ ]:
print("BR simples:")
BR_simples(X_train, y_train, X_test, y_test)

print("\nBR com Grid Search:")
BR_GS(X_train, y_train, X_test, y_test)

BR simples:
Hamming Loss = 0.16666666666666666
Acurácia = 0.2647058823529412
Precisão = 0.6764705882352942
Revocação = 0.2875

BR com Grid Search:
Melhores parâmetros = {'classifier': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'classifier__kernel': 'rbf'} 
Melhor score = 0.28852459016393445
Hamming Loss = 0.16666666666666666
Acurácia = 0.2647058823529412
Precisão = 0.6764705882352942
Revocação = 0.2875


In [ ]:
# MLkNN
print("MLkNN simples:")
MLkNN_simples(X_train, y_train, X_test, y_test)

print("\nMLkNN com Grid Search:")
MLkNN_GS(X_train, y_train, X_test, y_test)

MLkNN simples:
Hamming Loss = 0.18382352941176472
Acurácia = 0.13725490196078433
Precisão = 0.5449101796407185
Revocação = 0.37916666666666665

MLkNN com Grid Search:
Melhores parâmetros = {'k': 9, 's': 0.1} 
Melhor score = 0.2163934426229508
Hamming Loss = 0.16339869281045752
Acurácia = 0.23529411764705882
Precisão = 0.644927536231884
Revocação = 0.37083333333333335


In [ ]:
# LP
print("LP simples:")
LP_simples(X_train, y_train, X_test, y_test)

print("\nLP com Grid Search:")
LP_GS(X_train, y_train, X_test, y_test)

LP simples:
Hamming Loss = 0.16176470588235295
Acurácia = 0.2549019607843137
Precisão = 0.6721311475409836
Revocação = 0.3416666666666667

LP com Grid Search:
Melhores parâmetros = {'classifier': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'classifier__criterion': 'gini', 'classifier__n_estimators': 50} 
Melhor score = 0.28852459016393445
Hamming Loss = 0.16666666666666666
Acurácia = 0.2549019607843137
Precisão = 0.6578947368421053
Revocação = 0.3125


---
## Birds

explicação rápida do dataset aqui.

In [ ]:
df_birds_train = pd.read_csv("https://raw.githubusercontent.com/LeoRyu18/AM2_T1/main/csv_result-birds-train.csv")
df_birds_train.head()

,id,audio-ssd1,audio-ssd2,audio-ssd3,audio-ssd4,audio-ssd5,audio-ssd6,audio-ssd7,audio-ssd8,audio-ssd9,audio-ssd10,audio-ssd11,audio-ssd12,audio-ssd13,audio-ssd14,audio-ssd15,audio-ssd16,audio-ssd17,audio-ssd18,audio-ssd19,audio-ssd20,audio-ssd21,audio-ssd22,audio-ssd25,audio-ssd26,audio-ssd27,audio-ssd28,audio-ssd29,audio-ssd30,audio-ssd31,audio-ssd32,audio-ssd33,audio-ssd34,audio-ssd35,audio-ssd36,audio-ssd37,audio-ssd38,audio-ssd39,audio-ssd40,audio-ssd41,...,cluster89,cluster90,cluster91,cluster92,cluster93,cluster94,cluster95,cluster96,cluster97,cluster98,cluster99,cluster100,segments,mean_rect_width,std_rect_width,mean_rect_height,std_rect_height,mean_rect_volume,std_rect_volume,hasSegments,location,Brown,Pacific,Pacific-slope,Red-breasted,Dark-eyed,Olive-sided,Hermit,Chestnut-backed,Varied,Hermit.1,Swainson,Hammond,Western,Black-headed,Golden,Warbling,MacGillivray,Stellar,Common
0,1,0.016521,0.039926,0.089632,0.134119,0.170470,0.176872,0.171546,0.182392,0.162482,0.159083,0.164531,0.163366,0.171633,0.219787,0.270805,0.339206,0.327098,0.264581,0.173363,0.131426,0.068158,0.001216,0.000493,0.001114,0.003419,0.004479,0.004231,0.004013,0.003757,0.003314,0.003246,0.002308,0.002129,0.001777,0.002338,0.002030,0.003741,0.005809,0.006852,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,16.384615,20.617394,46.769231,71.863118,788.923077,1761.802180,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,2,0.006600,0.035984,0.089956,0.123214,0.172273,0.177068,0.165507,0.179655,0.161744,0.163678,0.161606,0.159523,0.171042,0.217206,0.254929,0.307129,0.293592,0.242930,0.151817,0.105817,0.062566,0.001245,0.000118,0.000972,0.003037,0.004070,0.004311,0.004704,0.003967,0.003847,0.002940,0.002346,0.002030,0.001950,0.002063,0.002207,0.002420,0.002521,0.001875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0.006894,0.017722,0.048062,0.065802,0.103443,0.091397,0.084931,0.088666,0.075676,0.074408,0.074683,0.083202,0.088820,0.125175,0.165580,0.212101,0.217109,0.153888,0.099709,0.074910,0.045928,0.001095,0.000125,0.000368,0.001433,0.002134,0.002286,0.001869,0.001417,0.001420,0.001047,0.000855,0.000764,0.000865,0.000837,0.001438,0.002761,0.005301,0.005453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,24.000000,2.828427,28.000000,1.414214,674.000000,113.137085,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,4,0.031046,0.127675,0.221428,0.272707,0.358743,0.349389,0.316029,0.330656,0.310752,0.306288,0.300054,0.304569,0.295422,0.367728,0.398225,0.457381,0.429034,0.330248,0.213530,0.131256,0.075369,0.001508,0.000894,0.005276,0.008511,0.010244,0.010371,0.009083,0.008407,0.006876,0.006512,0.005826,0.004873,0.004481,0.004114,0.004384,0.004423,0.004448,0.002896,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.064721,0.226644,0.304482,0.274662,0.346980,0.334063,0.307223,0.324666,0.297070,0.292258,0.287987,0.289535,0.286109,0.354479,0.389812,0.444778,0.407931,0.312861,0.197836,0.122053,0.069657,0.001287,0.002601,0.015984,0.014081,0.008860,0.009464,0.008694,0.007648,0.007977,0.006727,0.005021,0.004376,0.004248,0.004151,0.004636,0.004249,0.004037,0.002778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_birds_test = pd.read_csv("https://raw.githubusercontent.com/LeoRyu18/AM2_T1/main/csv_result-birds-test.csv")
df_birds_test.head()

,id,audio-ssd1,audio-ssd2,audio-ssd3,audio-ssd4,audio-ssd5,audio-ssd6,audio-ssd7,audio-ssd8,audio-ssd9,audio-ssd10,audio-ssd11,audio-ssd12,audio-ssd13,audio-ssd14,audio-ssd15,audio-ssd16,audio-ssd17,audio-ssd18,audio-ssd19,audio-ssd20,audio-ssd21,audio-ssd22,audio-ssd25,audio-ssd26,audio-ssd27,audio-ssd28,audio-ssd29,audio-ssd30,audio-ssd31,audio-ssd32,audio-ssd33,audio-ssd34,audio-ssd35,audio-ssd36,audio-ssd37,audio-ssd38,audio-ssd39,audio-ssd40,audio-ssd41,...,cluster89,cluster90,cluster91,cluster92,cluster93,cluster94,cluster95,cluster96,cluster97,cluster98,cluster99,cluster100,segments,mean_rect_width,std_rect_width,mean_rect_height,std_rect_height,mean_rect_volume,std_rect_volume,hasSegments,location,Brown,Pacific,Pacific-slope,Red-breasted,Dark-eyed,Olive-sided,Hermit,Chestnut-backed,Varied,Hermit.1,Swainson,Hammond,Western,Black-headed,Golden,Warbling,MacGillivray,Stellar,Common
0,1,0.132445,0.143931,0.227729,0.298556,0.385907,0.378363,0.354708,0.384165,0.360092,0.347465,0.341827,0.349941,0.349431,0.425508,0.470748,0.545955,0.515480,0.439492,0.287173,0.134652,0.082536,0.001778,0.009794,0.006053,0.008890,0.011114,0.010665,0.010292,0.008779,0.008474,0.008938,0.006323,0.006312,0.005450,0.005856,0.004785,0.005768,0.005577,0.004093,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0.101617,0.130342,0.228117,0.281017,0.365804,0.370122,0.359235,0.388608,0.362013,0.348229,0.342542,0.345851,0.338571,0.424733,0.470891,0.547948,0.516554,0.445265,0.293809,0.140007,0.088689,0.001953,0.005568,0.006061,0.008843,0.009781,0.009482,0.010100,0.011016,0.009763,0.007648,0.006345,0.005849,0.005672,0.005793,0.005648,0.005650,0.005195,0.003621,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,0.005148,0.017877,0.042137,0.062124,0.097340,0.088305,0.084337,0.083204,0.074532,0.071497,0.074953,0.077544,0.086848,0.126177,0.209470,0.275132,0.270673,0.238987,0.229216,0.194186,0.094694,0.001207,0.000084,0.000434,0.001294,0.001815,0.002196,0.001996,0.001737,0.001442,0.001275,0.000764,0.000722,0.000609,0.000805,0.001435,0.017121,0.022838,0.013595,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,17,27.235294,44.822526,91.647059,199.564231,2939.823529,9015.550592,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0
3,4,0.018792,0.012898,0.027330,0.039521,0.064671,0.068329,0.065799,0.059891,0.048287,0.047820,0.049324,0.055350,0.060587,0.086072,0.111716,0.150919,0.170447,0.133900,0.089051,0.068235,0.043178,0.001115,0.001177,0.000235,0.000642,0.000881,0.001309,0.001437,0.001096,0.000785,0.000561,0.000502,0.000439,0.000442,0.000457,0.000548,0.000812,0.000987,0.000992,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.007008,0.014610,0.033637,0.042604,0.065649,0.065047,0.064553,0.058155,0.048516,0.047021,0.050417,0.054324,0.059212,0.086488,0.110848,0.147765,0.163671,0.130540,0.088572,0.068011,0.043039,0.001082,0.000169,0.000279,0.000833,0.001016,0.001409,0.001123,0.001058,0.000874,0.000639,0.000493,0.000435,0.000443,0.000467,0.000554,0.000733,0.000856,0.000948,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
labels =  ['Brown', 'Pacific', 'Pacific-slope', 'Red-breasted', ' Dark-eyed', 'Olive-sided', 'Hermit', 'Chestnut-backed', 'Varied', 'Hermit', 'Swainson', 'Hammond', 'Western', 'Black-headed', 'Golden', 'Warbling', 'MacGillivray', 'Stellar', 'Common']

y_train_bird = np.array(df_birds_train[labels])
y_test_bird = np.array(df_birds_test[labels])

X_train = df_birds_train.drop(labels,axis=1)
X_test = df_birds_test.drop(labels,axis=1)

X_train = preprocessing.MinMaxScaler().fit_transform(X_train)
X_test = preprocessing.MinMaxScaler().fit_transform(X_test)

X_train_bird = np.array(X_train)
X_test_bird = np.array(X_test)



In [ ]:
for i in X_train_bird:
  for j in i:
    if j <0:
      print(j) 

In [ ]:
print("BR simples:")
BR_simples(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

print("\nBR com Grid Search:")
BR_GS(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

BR simples:
Hamming Loss = 0.04497311389929933
Acurácia = 0.5201238390092879
Precisão = 0.9428571428571428
Revocação = 0.10749185667752444

BR com Grid Search:
Melhores parâmetros = {'classifier': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'classifier__kernel': 'linear'} 
Melhor score = 0.5373076923076923
Hamming Loss = 0.04790614306664494
Acurácia = 0.5139318885448917
Precisão = 0.5286343612334802
Revocação = 0.39087947882736157


In [ ]:
# MLkNN
print("MLkNN simples:")
MLkNN_simples(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

print("\nMLkNN com Grid Search:")
MLkNN_GS(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

MLkNN simples:
Hamming Loss = 0.046928466677529736
Acurácia = 0.5294117647058824
Precisão = 0.553072625698324
Revocação = 0.32247557003257327

MLkNN com Grid Search:
Melhores parâmetros = {'k': 4, 's': 1.0} 
Melhor score = 0.5000961538461539
Hamming Loss = 0.046765520612677206
Acurácia = 0.5170278637770898
Precisão = 0.5735294117647058
Revocação = 0.254071661237785


In [ ]:
# LP
print("LP simples:")
LP_simples(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

print("\nLP com Grid Search:")
LP_GS(X_train_bird, y_train_bird, X_test_bird, y_test_bird)

LP simples:
Hamming Loss = 0.04024767801857585
Acurácia = 0.5356037151702786
Precisão = 0.8260869565217391
Revocação = 0.247557003257329

LP com Grid Search:
Melhores parâmetros = {'classifier': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'classifier__criterion': 'entropy', 'classifier__n_estimators': 50} 
Melhor score = 0.5559615384615385
Hamming Loss = 0.04383249144533159
Acurácia = 0.5510835913312694
Precisão = 0.6158536585365854
Revocação = 0.3289902280130293
